# Notebook DCC013

Iremos unir os arquivos da pasta planos-departamentais e situacao-turmas para construir um DataFrame e avaliar o desempenho dessas duas turmas

A questão a ser analisada será sobre a ocupação das turmas e o abandono da disiplina pelos alunos. Diminuir o número de vagas da turma estrutuda de dados, surtiu efeito na taxa de abandono da disciplina? Essa taxa caiu?

Importação das bibliotecas que serão usadas

In [ ]:
import pandas as pd 
import seaborn as sns 
import numpy as np
import matplotlib.pyplot as plt
from stackbar import mycatplot

Para imprimir todas as colunas e linhas retire o '#' da celula abaixo

In [ ]:
#pd.set_option('display.max_rows', None)
#pd.set_option('display.max_columns', None)

Aquivos que devam ser concertados, usar notebook correcao antes de rodá-los aqui

### Configurando o Data Frame

Se necessário acrescente anos na lista abaixo, para leitura de pastas de 2021 para frente.

In [ ]:
pasta = "situacao-turmas/"
modificado = "-mod"
anos = [2014, 2015, 2016, 2017, 2018, 2019, 2020]
periodos = ['1', '3']

Ler cada pasta e conecta os dados 

In [ ]:
df_anos = []
for ano in anos:
    df_periodos = []
    for periodo in periodos:
        df = pd.read_csv(pasta + str(ano) + "." + periodo + modificado + ".csv", sep = ';', 
                         encoding = 'ISO-8859-1', error_bad_lines=False, warn_bad_lines=True,
                        index_col = ['DISCIPLINA', 'TURMA'])
        df_periodos.append (df)
    df_anos.append(pd.concat(df_periodos, keys=periodos))

df_final = pd.concat (df_anos, keys=anos, names=['ANO', 'PERIODO', 'DISCIPLINA', 'TURMA']).reset_index()
display(df_final)

Separa o codigo do curso do nome do curso

In [ ]:
df = df_final['CURSO'].str.split(' - ', expand=True, n=1)
display(df)

In [ ]:
df_final.insert(loc=8, column='CODIGO_CURSO', value=df.iloc[:,0:1])
df_final.insert(loc=9, column='NOME_CURSO', value=df.iloc[:,1:2])
display(df_final)

Separa as disciplinas interessadas interessada. Substitua a disciplina entre aspas de acordo com a desejada

In [ ]:
codigo = 'DCC013'
one = df_final.query("DISCIPLINA == @codigo")
display(one)

### Gráficos de Vagas Oferecidas e Ocupação das Turmas

Retira linhas de vagas duplicadas e calcula a ocupação das turmas 

In [ ]:
vagas_totais = one[["ANO", "PERIODO", "DISCIPLINA", "TURMA", "VAGAS OFERECIDAS", "VAGAS OCUPADAS"]].drop_duplicates()
vagas_totais["OCUPACAO EM PORCENTAGEM"] = (vagas_totais["VAGAS OCUPADAS"] / vagas_totais["VAGAS OFERECIDAS"])*100
display(vagas_totais)

Coloca a grade atras dos gráficos

In [ ]:
sns.set_style("whitegrid")
sns.set_context("paper", font_scale=1.5) 

Grafico das vagas oferecidas por disciplina de acordo com o ano e periodo

In [ ]:
a = sns.catplot(data=vagas_totais, x="ANO", y="VAGAS OFERECIDAS", kind="bar", hue="DISCIPLINA", col="PERIODO",
               palette=['#8ECCB7'], estimator=np.sum, ci=None)
a.set_titles("Vagas Oferecidas no Período {col_name}", fontdict= {'fontweight':'semibold'})
a.savefig("VagasOferecidas.pdf")
a.savefig("VagasOferecidas.png")

Grafico vagas oferecidas das disciplinas, em linha

In [ ]:
c = sns.relplot(data=vagas_totais, x="ANO", y="VAGAS OFERECIDAS", kind="line", col="PERIODO",
            hue="DISCIPLINA", estimator=np.sum, palette=['#8ECCB7'])
c.set(ylim = (0,200))
c.set(xlim = (2014,2020))
c.despine(left=True)
c.set_titles("Vagas Oferecidas no Período {col_name}", fontdict= {'fontweight':'semibold'})

Grafico da ocupação das turmas por disciplina de acordo com o ano e periodo

In [ ]:
b = sns.catplot(data=vagas_totais, x="ANO", y="OCUPACAO EM PORCENTAGEM", palette=['#8ECCB7'], 
                kind="bar", hue="DISCIPLINA", col="PERIODO", ci=None)
b.set_titles("Ocupação das Turmas Período {col_name}", fontdict= {'fontweight':'bold'})
b.savefig("UsoDaTurma.pdf")
b.savefig("UsoDaTurma.png")

Grafico da ocupação das disciplinas em linha 

In [ ]:
d = sns.relplot(data=vagas_totais, x="ANO", y="OCUPACAO EM PORCENTAGEM", kind="line", col="PERIODO",
            hue="DISCIPLINA", palette=['#CC527A', '#2F9395'])
d.set(ylim = (0,100))
d.set(xlim = (2014,2020))
d.despine(left=True)
d.set_titles("Ocupação das Turmas Período {col_name}", fontdict= {'fontweight':'bold'})

### Gráficos dos alunos por curso

In [ ]:
por_curso = one.groupby(["ANO", "PERIODO", "DISCIPLINA", "TURMA","CODIGO_CURSO", "NOME_CURSO"])[["TOTAL DE ALUNOS"]].sum().reset_index()
display(por_curso)

In [ ]:
e = sns.catplot(data=por_curso, x="ANO", y="TOTAL DE ALUNOS", row = "CODIGO_CURSO", kind="bar", hue="DISCIPLINA",
           row_order=['35A', '76A', '65AC', '65AB', '65A'], estimator=np.sum, ci=None,
            palette=['#9CD3C0', '#FF9999'])
e.fig.set_figheight(20)
e.fig.set_figwidth(25)
e.set_titles("Total de Alunos nas Disciplinas do Curso {row_name}", fontdict= {'fontweight':'bold'})

In [ ]:
f= sns.catplot(data=por_curso, x="ANO", y="TOTAL DE ALUNOS", kind="bar", hue="DISCIPLINA", col="PERIODO",
           estimator=np.sum, ci=None, palette=['#9CD3C0', '#FF9999'] )
f.set_titles("Total de Alunos nas Turmas no Período {col_name}", fontdict= {'fontweight':'bold'})

### Gráficos sobre a situação dos alunos 

In [ ]:
situacao1 = one.groupby(["ANO", "PERIODO", "DISCIPLINA"])[["TOTAL DE ALUNOS"]].sum().reset_index()
display(situacao1)

In [ ]:
def total_alunos(ano, periodo, disciplina):
    return situacao1.query("ANO == @ano and PERIODO == @periodo and DISCIPLINA == @disciplina").iloc[0]["TOTAL DE ALUNOS"]

In [ ]:
situacao2 = one.groupby(["ANO", "PERIODO", "DISCIPLINA", "SITUACAO"])[["TOTAL DE ALUNOS"]].sum().reset_index()
display(situacao2)

In [ ]:
g = sns.catplot(data=situacao2, x="ANO", y="TOTAL DE ALUNOS", kind="bar", row="SITUACAO", hue="DISCIPLINA",
            col="PERIODO", estimator=np.sum, ci=None, palette=['#9CD3C0', '#FF9999'])
g.set_titles("Situação dos Alunos {row_name} no Período {col_name}",  fontdict= {'fontweight':'bold'})

In [ ]:
situacao2["CALCULO_ALUNOS"] = 0
for index, row in situacao2.iterrows():
    calculo = (row['TOTAL DE ALUNOS']/(total_alunos(row["ANO"], row["PERIODO"], row["DISCIPLINA"])))*100
    situacao2.loc[index, 'CALCULO_ALUNOS'] = calculo
situacao2["ANOPERIODO"] = situacao2["ANO"].astype(str)+"."+situacao2["PERIODO"].astype(str)     
display(situacao2)

In [ ]:
i = sns.catplot(data=situacao2, x="ANO", y="CALCULO_ALUNOS", kind="bar", row="SITUACAO", hue="DISCIPLINA", 
            col="PERIODO", estimator=np.sum, ci=None, palette=['#9CD3C0', '#FF9999'])
i.set_titles("Situação dos Alunos {row_name} no Período {col_name}", fontdict= {'fontweight':'bold'})

In [ ]:
j =sns.catplot(data=situacao2, x="ANO", y="CALCULO_ALUNOS", kind="bar", row="SITUACAO",
            hue="DISCIPLINA", estimator=np.sum, ci=None, palette=['#9CD3C0', '#FF9999'])
j.set_titles("Situação dos Alunos {row_name}")

In [ ]:
h= mycatplot(x="ANOPERIODO", y="CALCULO_ALUNOS", data=situacao2, kind="bar", aspect=1.7,
         ci=None, hue="SITUACAO",
          col="DISCIPLINA", palette=['#2A9D8F','#F4A261', '#D7431D', '#A8A7A8', '#474747'])
sns.set(rc = {'figure.figsize':(20,8)})
h.set_titles("Situação Final dos Alunos em {col_name}", fontdict= { 'fontsize': 30, 'fontweight':'bold'})
h.savefig("Situacao.pdf")
h.savefig("Situacao.png")